Following the results obtained in the previous notebook (pipeline_harry_potter.ipynb), i will try to utilize a generative model to answer the questions

I will start with the usual setup for Haystack

In [1]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss,ocr]

     |████████████████████████████████| 2.1 MB 5.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-8pvopg_w/farm-haystack_87282df42c8346d0aa3d6c1a77d4f1d0
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-8pvopg_w/farm-haystack_87282df42c8346d0aa3d6c1a77d4f1d0
  Resolved https://github.com/deepset-ai/haystack.git to commit ae712fe6bf087c717f3e38e4e87d2347165fc12b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [ ]:
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz
!tar -xvf xpdf-tools-linux-4.03.tar.gz && sudo cp xpdf-tools-linux-4.03/bin64/pdftotext /usr/local/bin

Setting up FAISS as document store

In [3]:
from haystack.document_stores import FAISSDocumentStore
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)


INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


Let's convert the pdf and apply the preprocessing routine

In [49]:
# Here are the imports we need
from haystack.nodes import PDFToTextConverter, PreProcessor
from haystack.utils import convert_files_to_docs

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_pdf = converter.convert(file_path="/content/Harry Potter and The Sorcerer’s Stone.pdf", meta=None)[0]
doc_pdf_2 = converter.convert(file_path="/content/HP-chamber-of-secret.pdf", meta=None)[0]
doc_pdf_3 = converter.convert(file_path="/content/Harry-potter-prison-of-azkaban.pdf", meta=None)[0]
doc_pdf_4 = converter.convert(file_path="/content/HP-goblet-of-fire.pdf", meta=None)[0]
doc_pdf_5 = converter.convert(file_path="/content/harry-potter-and-the-order-of-the-phoenix.pdf", meta=None)[0]
doc_pdf_6 = converter.convert(file_path="/content/HP-half-blood-prince.pdf", meta=None)[0]
doc_pdf_7 = converter.convert(file_path="/content/Harry-Potter-and-the-Deathly-Hallows.pdf", meta=None)[0]



preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process([doc_pdf, doc_pdf_2, doc_pdf_3, doc_pdf_4, doc_pdf_5, doc_pdf_6, doc_pdf_7])
print(f"n_docs_input: 1\nn_docs_output: {len(docs)}")

100%|██████████| 7/7 [00:03<00:00,  2.08docs/s]

n_docs_input: 1
n_docs_output: 12556


In [50]:
from haystack import Document
from typing import List

documents: List[Document] = []

for doc in docs:
    documents.append(Document(content=doc.content, meta={"name": "Harry Potter"}))

Initialize the DensePassageRetriever and RAGenerator and update the embedding in the document store

In [36]:
from haystack.nodes import RAGenerator, DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  ===

In [37]:
from haystack.nodes import RAGenerator, DensePassageRetriever


# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=200,
    min_length=2,
    embed_title=True,
    num_beams=2,
)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
/usr/local/lib/python3.7/dist-packages/transformers/models/bart/configuration_bart.py:178: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  f"Please make sure the config includes `forced_bos_token_id={self.bos_token_id}` in future versions. "
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this

Add the document in FAISS

In [51]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)

Writing Documents:   0%|          | 0/12556 [00:00<?, ?it/s]

INFO - haystack.document_stores.faiss -  Updating embeddings for 12546 docs...


Updating Embedding:   0%|          | 0/12546 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/2560 [00:00<?, ? Docs/s]

In [77]:
questions = [
             'Who is Dumbledore?', 
             "Who are Harry Potter's parents?", 
             "what are the names of the four houses?", 
             "Is Hermione Harry Potter's best friend?", 
             "is harry potter's house-elf called dobby?"
             ]

In [78]:
from haystack.utils import print_answers
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
for question in questions:
    res = pipe.run(query=question, params={"Generator": {"top_k": 1}, "Retriever": {"top_k": 10}})
    print_answers(res, details="minimum")


/usr/local/lib/python3.7/dist-packages/transformers/models/rag/tokenization_rag.py:97: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1749: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  UserWarning,



Query: Who is Dumbledore?
Answers:
[{'answer': ' albus dumbledore'}]

Query: Who are Harry Potter's parents?
Answers:
[{'answer': ','}]

Query: what are the names of the four houses?
Answers:
[{'answer': ' salazar slytherin'}]

Query: Is Hermione Harry Potter's best friend?
Answers:
[{'answer': ' colin creevey'}]

Query: is harry potter's house-elf called dobby?
Answers:
[{'answer': ' the'}]


Terrible results. Let's try with LFQA in order to have better answers

In [40]:
from haystack.nodes import Seq2SeqGenerator
generator_ = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [54]:
from haystack.pipelines import GenerativeQAPipeline

pipe_ = GenerativeQAPipeline(generator_, retriever)

In [79]:
for question in questions:
    res = pipe_.run(query=question, params={"Retriever": {"top_k": 5}})
    print_answers(res, details="minimum")


Query: Who is Dumbledore?
Answers:
[   {   'answer': 'Albus Dumbledore is the headmaster of Hogwarts School of '
                  'Witchcraft and Wizardry. He is responsible for the '
                  "discovery of the twelve uses of dragon's blood, and his "
                  'work on alchemy with his partner, Nicolas Flamel.'}]

Query: Who are Harry Potter's parents?
Answers:
[   {   'answer': "I'm not sure if this is what you're looking for, but I'll "
                  "give it a shot. Harry's parents are the Dursleys. They were "
                  'Muggles who hated and despised magic, and as far as they '
                  'were concerned, having a wizard in the family was a matter '
                  'of deepest shame.'}]

Query: what are the names of the four houses?
Answers:
[   {   'answer': 'Gryffindor, Hufflepuff, Ravenclaw, and Salazar Slytherin. '
                  'Each house has its own noble history and each has produced '
                  'outstanding witches and 

I am incredibly impressed by the results. In many cases it sounds very natural. After running many tests, it is clear that there are many problems but already in this way the results are noticeably. For example, the last answer sounds pretty unnaturale.